In [20]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Preprocess / Load / Split Data

In [21]:
df = pd.read_csv('labelled_images.csv')
df.head()

,Path,Target
0,datasets/Alex/Alex-Image119.png,Alex
1,datasets/Alex/Alex-Image131.png,Alex
2,datasets/Alex/Alex-Image125.png,Alex
3,datasets/Alex/Alex-Image247.png,Alex
4,datasets/Alex/Alex-Image27.png,Alex


In [22]:
# Function to load and preprocess images
def preprocess_images(df, target_size=(128, 128)):
    images = []
    labels = []
    
    for index, row in df.iterrows():
        # Load image and resize
        image = load_img(row['Path'], target_size=target_size)
        image = img_to_array(image)  # Convert to numpy array
        image = image / 255.0  # Normalize pixel values to [0, 1]
        
        # Append the image and its corresponding label
        images.append(image)
        labels.append(row['Target'])

    return np.array(images), np.array(labels)

X, y = preprocess_images(df)
print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")


Shape of X: (485, 128, 128, 3), Shape of y: (485,)


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")

Training set shape: (388, 128, 128, 3), (388,)
Test set shape: (97, 128, 128, 3), (97,)


In [24]:
# Encode labels (Alex -> 0, Kelly -> 1)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Check the encoded labels
print(f"Encoded training labels: {y_train_encoded[:10]}")


Encoded training labels: [1 0 0 0 1 1 0 1 1 1]


# Convolutional Neural Network (Images Only)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [26]:
# Build the CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the data and add dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer with a sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test, y_test_encoded))

# Print the training and validation accuracy after training
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - accuracy: 0.5208 - loss: 0.9808 - val_accuracy: 0.4742 - val_loss: 0.6966
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 153ms/step - accuracy: 0.5325 - loss: 0.6755 - val_accuracy: 0.5979 - val_loss: 0.6596
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.6213 - loss: 0.6271 - val_accuracy: 0.7113 - val_loss: 0.6255
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 0.7129 - loss: 0.5882 - val_accuracy: 0.5773 - val_loss: 0.7092
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 0.7601 - loss: 0.5308 - val_accuracy: 0.6804 - val_loss: 0.6408
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.8600 - loss: 0.3840 - val_accuracy: 0.5979 - val_loss: 0.7400
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 0.8615 - loss: 0.3603 - val_accuracy: 0.6186 - val_loss: 0.7345
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - accuracy: 0.8921 - loss: 0.3011 - val_accuracy: 0.6701 - val_

test accuracy pretty bad :( --> overfitting again

ideas to fix overfitting
- increase regularization
- data augmentation? -- rotations, flips, zooms
- reduce model complexity
- early stopping
- different learning rate
- external data
- cross val

In [ ]:
# try gridsearch later??

# CNN with everything?

In [41]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split


# Load the CSV containing the features
features_df = pd.read_csv('images.csv')


# Separate the features and target variable
X = features_df.drop(columns=['Image_Name', 'Target', 'Path'])
y = features_df['Target']


# Train/test split for the features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


# Load images from file paths in the 'Path' column
def load_and_preprocess_images(image_paths, image_size=(128, 128)):
   images = []
   for path in image_paths:
       try:
           img = load_img(path, target_size=image_size)  # Load and resize the image
           img_array = img_to_array(img) / 255.0  # Convert image to array and normalize
           images.append(img_array)
       except FileNotFoundError:
           print(f"File not found: {path}")
           images.append(np.zeros((image_size[0], image_size[1], 3)))  # Add a blank image as a placeholder
   return np.array(images)


# Assuming the images are stored in the file paths in the 'Path' column
X_image_train = load_and_preprocess_images(features_df.loc[X_train.index, 'Path'])
X_image_test = load_and_preprocess_images(features_df.loc[X_test.index, 'Path'])


# CNN Model for image data
model = Sequential()


# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# Flatten the data and add dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))


# Output layer with a sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit(X_image_train, y_train, epochs=10, batch_size=32, validation_data=(X_image_test, y_test))


# Print the training and validation accuracy after training
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")


# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_image_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'Kelly-Image91.png'

In [42]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


# Function to build the model with hyperparameters
def build_model(hp):
   model = Sequential()


   # Convolutional layers with hyperparameters for filters and kernel size
   model.add(Conv2D(
       hp.Int('filters1', min_value=32, max_value=128, step=32),
       (3, 3),
       activation='relu',
       input_shape=(128, 128, 3)
   ))
   model.add(MaxPooling2D(pool_size=(2, 2)))
  
   model.add(Conv2D(
       hp.Int('filters2', min_value=64, max_value=256, step=64),
       (3, 3),
       activation='relu'
   ))
   model.add(MaxPooling2D(pool_size=(2, 2)))
  
   # Flatten and Dense layers with hyperparameters
   model.add(Flatten())
   model.add(Dense(
       hp.Int('dense_units', min_value=64, max_value=256, step=64),
       activation='relu'
   ))
   model.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))
  
   # Output layer with sigmoid activation for binary classification
   model.add(Dense(1, activation='sigmoid'))
  
   # Compile the model with a hyperparameter for learning rate
   model.compile(
       optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')),
       loss='binary_crossentropy',
       metrics=['accuracy']
   )
  
   return model

,Unnamed: 0,Has_People,People_Count,Has_Building,Building_Count,Animal_Type,Animal_Subtype,Is_Landscape_Nature,Has_BoardGames,Has_Road_Pathway,Is_Light_Dark,Is_Close_Far,Is_Straight_On,Has_Horizon,Orientation,Is_Sunrise_Sunset,Has_Tent_Campfire,Has_Food
460,460,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
25,25,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
220,220,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0
234,234,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0
357,357,1,1,1,1,1,2,0,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,1,6,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
270,270,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0
348,348,1,2,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0
435,435,1,2,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0


In [ ]:
# Instantiate a RandomSearch tuner with the model-building function
tuner = kt.RandomSearch(
   build_model,
   objective='val_accuracy',
   max_trials=5,  # Number of different combinations to test
   executions_per_trial=3,  # Number of times to train the model with each combination
   directory='my_dir',  # Directory to save the search results
   project_name='image_classification'
)


# Search for the best hyperparameters
tuner.search(X_image_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_image_test, y_test_encoded))

In [ ]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

In [ ]:
# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)


# Train the model with the best hyperparameters
history = model.fit(X_image_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_image_test, y_test_encoded))

In [ ]:
test_loss, test_acc = model.evaluate(X_image_test, y_test_encoded)
print(f"Test Accuracy: {test_acc:.4f}")

a little better, but can still improve

- regularization
- more epochs, lower learning rate?
- cv?